In [13]:
%pwd

'/home/user/Documents/ML DL PROJECTS/ChickenDiseaseClassificationDLProject'

In [14]:
%cd /home/user/Documents/ML DL PROJECTS/ChickenDiseaseClassificationDLProject/

/home/user/Documents/ML DL PROJECTS/ChickenDiseaseClassificationDLProject


In [15]:
%pwd

'/home/user/Documents/ML DL PROJECTS/ChickenDiseaseClassificationDLProject'

In [16]:
import tensorflow as tf

In [17]:
model = tf.keras.models.load_model("artifacts/training/model.h5")
from pathlib import Path

In [18]:
from dataclasses import dataclass
@dataclass(frozen=True)
class ModelEvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int 

In [19]:
from ChickenDiseaseClassification.constants import *
from ChickenDiseaseClassification.utils.common import read_yaml,create_directories,save_json


In [20]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root]) 
    def GetModelEvaluationConfig(self)->ModelEvaluationConfig:
        model_evaluation_config = ModelEvaluationConfig(path_of_model="artifacts/training/model.h5", training_data="artifacts/data_ingestion/Chicken-fecal-images",all_params=self.params,params_image_size=self.params.IMAGE_SIZE,params_batch_size=self.params.BATCH_SIZE)
        return model_evaluation_config

In [21]:
from urllib.parse import  urlparse

In [22]:
class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH,params_file_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root]) 
    def GetModelEvaluationConfig(self)->ModelEvaluationConfig:
        model_evaluation_config = ModelEvaluationConfig(path_of_model="artifacts/training/model.h5", training_data="artifacts/data_ingestion/Chicken-fecal-images",all_params=self.params,params_image_size=self.params.IMAGE_SIZE,params_batch_size=self.params.BATCH_SIZE)
        return model_evaluation_config

In [23]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [24]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.GetModelEvaluationConfig()
    model_evaluation  = ModelEvaluation(model_evaluation_config)
    model_evaluation.evaluation()
    model_evaluation.save_score()
except Exception as e:
    raise e


[2023-10-10 16:13:39,071: INFO: common yaml file: config/config.yaml loaded succuessfully]
[2023-10-10 16:13:39,077: INFO: common yaml file: params.yaml loaded succuessfully]
[2023-10-10 16:13:39,078: INFO: common directory at artifacts created succuessfully]
Found 116 images belonging to 2 classes.


2023-10-10 16:13:40.343019: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-10-10 16:13:40.909113: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2023-10-10 16:13:43.094485: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 102760448 exceeds 10% of free system memory.
2023-10-10 16:13:43.221923: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.
2023-10-10 16:13:44.071345: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 205520896 exceeds 10% of free system memory.


4/4 [==============================] - 61s 15s/step - loss: 0.6275 - accuracy: 0.9224
[2023-10-10 16:14:40,899: INFO: common json file saved at : scores.json]
